# Exploration of Car Make and Violation Description Data from The City of Los Angeles Parking Citation Open Dataset

## Data cleanliness

Building on previous explorations of the Los Angeles Parking Citation Open Dataset, these analyses will further explore the connections between car make and parking violation type. Before going much further, data completeness and consistency has to be explored.

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import os
from pathlib import Path
import random
import seaborn as sns
from fuzzywuzzy import process

# Load project directory
PROJECT_DIR = Path(os.path.abspath('../..'))

In [2]:
df = pd.read_csv(PROJECT_DIR / 'data/raw/2021-01-02_raw.csv',skiprows=lambda i: i > 0 and random.random() > .01,)
df.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,...,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude,Agency Description,Color Description,Body Style Description
0,1107179581,12/27/2015,1055.0,NaN,NaN,CA,201605.0,NaN,TOYO,PA,...,NaN,54.0,8058L,PREF PARKING,68.0,99999.0,99999.0,NaN,NaN,NaN
1,1110265251,12/16/2015,1340.0,NaN,NaN,CA,NaN,NaN,NaN,TR,...,22MP4,1.0,5204A,EXPIRED TAGS,25.0,99999.0,99999.0,NaN,NaN,NaN
2,1112716673,12/28/2015,1020.0,NaN,NaN,CA,201601.0,NaN,KIA,PA,...,00461,54.0,8069BS,NO PARK/STREET CLEAN,73.0,99999.0,99999.0,NaN,NaN,NaN
3,1112718025,12/28/2015,1222.0,NaN,NaN,CA,201510.0,NaN,SATU,PA,...,00461,54.0,5204A,EXPIRED TAGS,25.0,99999.0,99999.0,NaN,NaN,NaN
4,1113965031,12/24/2015,1108.0,NaN,NaN,CA,201701.0,NaN,FORD,PA,...,00141,51.0,8069BS,NO PARK/STREET CLEAN,73.0,6436025.9,1833425.9,NaN,NaN,NaN


In [3]:
# Missing data
df[['Violation code', 'Violation Description']].isna().sum()/len(df)

Violation code           0.004298
Violation Description    0.009355
dtype: float64

In [4]:
# Unique pairs of data with missing data
df[['Violation code', 'Violation Description']][df['Violation code'].isna()|df['Violation Description'].isna()].drop_duplicates()[:10]

,Violation code,Violation Description
4714,000,NaN
98690,NaN,NaN
99242,8069AP,NaN
99244,8069A,NaN
99248,4000A1,NaN
99249,22514,NaN
99255,8058L,NaN
99335,8056E4,NaN
99338,8813B,NaN
99419,225078A,NaN


In [5]:
# Remove entries with both violation code and violation description missing
df = df[~(df['Violation code'].isna() & df['Violation Description'].isna())]

In [6]:
same_codes = set(df['Violation code']).intersection(set(df['Violation Description']))

df[['Violation code', 'Violation Description']][df['Violation Description'].isin(same_codes)]

,Violation code,Violation Description
194,024,22514
332,010,22500E
1080,024,22514
1191,098,5200
1370,013,22500H
...,...,...
103026,013,22500H
104846,011,22500F
104908,010,22500E
104909,098,5200


It would seem that there are a few violation codes that have been entered in as violation descriptions. The codes should be moved over and the descriptions should be deleted. It would seem that the 3 numeral violation codes are not very meaningful sometimes and that 000 is often used for different types violations.

In [7]:
# Create function to swap codes and descriptions
def code_swap(df):
    df['Violation code'] = df['Violation Description']
    df['Violation Description'] = np.nan
    return df

In [8]:
code_swap_filter = (df['Violation Description'].isin(same_codes) | (df['Violation code'] == '000'))

df.loc[code_swap_filter,['Violation code', 'Violation Description']] = df[['Violation code', 'Violation Description']][code_swap_filter].apply(code_swap, axis=1)

# Remove new entries with both violation code and violation description missing
df = df[~(df['Violation code'].isna() & df['Violation Description'].isna())]

In [9]:
df[['Violation code', 'Violation Description']].drop_duplicates().sort_values('Violation code')

,Violation code,Violation Description
4648,022,225078
22083,030,22522
13751,031,22523A
8908,032,22523B
274,099,5204
...,...,...
42623,8939B,NaN
104682,8940,NaN
622,8940,PARKING AREA
42089,8940B,PK OVR 2 SPACES


In [26]:
df['Violation Description'].value_counts()[:50]

NO PARK/STREET CLEAN              31520
METER EXP.                        19736
RED ZONE                           9008
PREFERENTIAL PARKING               7742
DISPLAY OF TABS                    6423
NO PARKING                         4886
DISPLAY OF PLATES                  2976
WHITE ZONE                         2312
PARKED OVER TIME LIMIT             2133
NO STOP/STANDING                   2070
STANDNG IN ALLEY                   1632
BLOCKING DRIVEWAY                  1507
PARKED OVER TIME LIM               1373
STOP/STAND PROHIBIT                1170
YELLOW ZONE                        1140
NO STOP/STAND                      1095
PARKED ON SIDEWALK                 1043
18 IN. CURB/2 WAY                  1012
NO EVIDENCE OF REG                  986
FIRE HYDRANT                        964
EXCEED 72HRS-ST                     950
NO STOPPING/ANTI-GRIDLOCK ZONE      840
DOUBLE PARKING                      803
OFF STR/OVERTIME/MTR                642
OUTSIDE LINES/METER                 576


In [67]:
df.loc[~df['Violation Description'].isin(["PREF PARKING", np.nan]), 'Violation Description'].unique()

array(['EXPIRED TAGS', 'NO PARK/STREET CLEAN', 'RED ZONE',
       'DISPLAY OF PLATES', 'DOUBLE PARKING', 'METER EXP.',
       'PREFERENTIAL PARKING', 'STOP/STAND PROHIBIT',
       'PARKED OVER TIME LIMIT', 'NO PARKING', '18 IN. CURB/2 WAY',
       'DISPLAY OF TABS', 'WHITE ZONE', 'PARKED ON SIDEWALK',
       'NO EVIDENCE OF REG', 'PARK FIRE LANE', 'DSPLYPLATE A',
       'EXCEED 72 HOURS', 'ANGLE PKD', 'DISABLED PARKING/CRO',
       'NO STOPPING/ANTI-GRIDLOCK ZONE', 'DISABLED PARKING/NO',
       'SAFETY ZONE/CURB', 'BLOCKING DRIVEWAY', 'STANDNG IN ALLEY',
       'PARKING/FRONT YARD', 'OFF STR/OVERTIME/MTR', 'HANDICAP/NO DP ID',
       'STNDNG IN ALLEY', 'FIRE HYDRANT', 'EXCEED 72HRS-ST',
       'YELLOW ZONE', 'OUTSIDE LINES/METER', '5204', 'PRIVATE PROPERTY',
       'PARK IN GRID LOCK ZN', 'COMM VEH OVER TIME LIMIT',
       'DP-BLKNG ACCESS RAMP', 'PK IN PROH AREA', '3 FT. SIDEWALK RAMP',
       'OVNIGHT PRK W/OUT PE', 'NO PARKING BETWEEN POSTED HOURS',
       'PARKED IN BUS ZONE', 'WHI

In [71]:
for phrase in df.loc[~df['Violation Description'].isin([phrase, np.nan]), 'Violation Description'].unique():
    print(pd.DataFrame(process.extract(phrase, df.loc[~df['Violation Description'].isin([phrase, np.nan]), 'Violation Description'].unique()), columns=[phrase, "Score"]), '\n')

    86
4            PREFERENTIAL PARKING     86 

               PARKED IN BUS ZONE  Score
0                        RED ZONE     86
1                      WHITE ZONE     86
2  NO STOPPING/ANTI-GRIDLOCK ZONE     86
3                     YELLOW ZONE     86
4                      GREEN ZONE     86 

            WHITE CURB  Score
0    18 IN. CURB/2 WAY     86
1     SAFETY ZONE/CURB     86
2  18 IN/CURB/COMM VEH     86
3    18 IN. CURB/1 WAY     86
4             RED CURB     67 

           PARKING AREA  Score
0   R/PRIV PARKING AREA     90
1  PREFERENTIAL PARKING     86
2  DISABLED PARKING/CRO     86
3   DISABLED PARKING/NO     86
4    PARKING/FRONT YARD     86 

          METER EXPIRED  Score
0            METER EXP.     82
1          EXPIRED TAGS     70
2              RED ZONE     53
3   OUTSIDE LINES/METER     53
4  MT FIRE RD NO PERMIT     53 

  18 IN/CURB/COMM VEH  Score
0          WHITE CURB     86
1            RED CURB     86
2        MORE 18-CURB     86
3   18 IN. CURB/2 WAY     73

In [10]:
code_dict = {}
for code in set(df['Violation code']):
    desc_aliases = df.loc[(df['Violation code'] == code), 'Violation Description'].drop_duplicates()        .dropna().to_list()
    if desc_aliases:
        if len(desc_aliases) > 1:
            code_dict[code] = max(desc_aliases, key=len)
        else:
            code_dict[code] = desc_aliases[0]

In [34]:
df['Violation Description'].value_counts()[:60]

NO PARK/STREET CLEAN               31559
METER EXP.                         19736
RED ZONE                            9105
PREFERENTIAL PARKING                7742
DISPLAY OF TABS                     6423
NO PARKING                          4918
STOP/STAND PROHIBIT                 4335
PARKED OVER TIME LIMIT              3506
DISPLAY OF PLATES                   3018
WHITE ZONE                          2315
STANDNG IN ALLEY                    1632
BLOCKING DRIVEWAY                   1558
NO STOPPING/ANTI-GRIDLOCK ZONE      1237
YELLOW ZONE                         1143
PARKED ON SIDEWALK                  1117
18 IN. CURB/2 WAY                   1065
NO EVIDENCE OF REG                  1059
FIRE HYDRANT                        1041
EXCEED 72HRS-ST                      950
DOUBLE PARKING                       927
OFF STR/OVERTIME/MTR                 642
OUTSIDE LINES/METER                  576
COMM VEH OVER TIME LIMIT             481
EXPIRED TAGS                         322
DISABLED PARKING

In [23]:
df.loc[df['Violation Description'].isin(['NO STOP/STANDING', 'STOP/STAND PROHIBIT', 'NO STOP/STAND']), ['Violation code', 'Violation Description']].drop_duplicates()

,Violation code,Violation Description
16,80.69AA+,NO STOP/STAND
25,80.69AP+,NO STOP/STANDING
80,80.69A+,STOP/STAND PROHIBIT


These 3 violations are different in very subtle ways but are essentially the same [law](https://codelibrary.amlegal.com/codes/los_angeles/latest/lamc/0-0-0-166554) so they will all be turned into STOP/STAND PROHIBIT by changing the dictionary that was already created and added to the make_data.py script:

"80.69AP+": "STOP/STAND PROHIBIT", #Changed from 'NO STOP/STANDING'
"80.69AA+": "STOP/STAND PROHIBIT", #Changed from "NO STOP/STAND"

In [27]:
code_dict.update({"80.69AP+": "STOP/STAND PROHIBIT","80.69AA+": "STOP/STAND PROHIBIT"})

In [31]:
for key, value in code_dict.items():
    df.loc[df["Violation code"] == key, "Violation Description"] = value
